In [ ]:
import pandas as pd
import requests
import os
import time
import random

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

In [ ]:
HEADERS = {"User-Agent": "Mozilla/5.0"}

# 0. 나스닥100 기업들의 티커
def get_nasdaq100_tickers(yahoo_style: bool = False) -> list[str]:
    
    url = "https://en.wikipedia.org/wiki/Nasdaq-100"
    html = requests.get(url, headers=HEADERS, timeout=20).text
    tables = pd.read_html(html)

    # 'Ticker' 또는 'Symbol' 컬럼이 포함된 표 찾기
    for t in tables:
        if any(col in t.columns for col in ["Ticker", "Symbol"]):
            df = t
            break
    else:
        raise RuntimeError("위키피디아에서 티커 표를 찾지 못했습니다.")

    col = "Ticker" if "Ticker" in df.columns else "Symbol"
    tickers = df[col].dropna().astype(str).tolist()

    # yahoofinance에서 검색가능한 형태로 형식을 바꿔줌
    if yahoo_style:
        tickers = [t.replace(".", "-") for t in tickers]

    return tickers

nasdaq100 = get_nasdaq100_tickers(yahoo_style=True)

In [ ]:
# 1. 티커에 대한 뉴스 url 요청
def get_href(query, count=20):
    url = f"https://query1.finance.yahoo.com/v1/finance/search?q={query}&newsCount={count}&start=0"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                      "AppleWebKit/537.36 (KHTML, like Gecko) "
                      "Chrome/127.0.0.0 Safari/537.36",
        "Accept": "application/json, text/javascript, */*; q=0.01",
        "Accept-Language": "en-US,en;q=0.9",
        "Referer": "https://finance.yahoo.com/",
        "Connection": "keep-alive",
    }

    response = requests.get(url, headers=headers)
    print(f"[{query}] Status Code: {response.status_code}")

    results = []
    if response.status_code == 200:
        data = response.json()
        for item in data.get("news", []):
            link = item.get("link")
            results.append(link)

    return results[:count]

In [ ]:
# 2. CSV 업데이트 함수

def update_csv(query, csv_path="yahoo_news.csv", filtered_path="yahoo_filtered.csv"):
    new_links = get_href(query)

    # 이전에 크롤링한 data
    if os.path.exists(csv_path):
        df_old = pd.read_csv(csv_path)
        old_pairs = set(zip(df_old["ticker"], df_old["link"]))
    else:
        df_old = pd.DataFrame(columns=["ticker", "link", "headline", "pubdate", "related_tickers", "article"])
        old_pairs=set()
    
    # 추출 대상에서 제외된 url data
    if os.path.exists(filtered_path):
        df_filtered = pd.read_csv(filtered_path)
        filtered_pairs = set(zip(df_filtered["ticker"], df_filtered["link"]))
    else:
        filtered_pairs = set()

    # 이전에 실행하지 않았던 url만 모으기
    unique_links = [
        link for link in new_links
        if (query, link) not in old_pairs and (query, link) not in filtered_pairs
    ]

    # 새로운 데이터프레임 생성 (티커 포함)
    df_new = pd.DataFrame(unique_links, columns=["link"])
    df_new["ticker"] = query

    # 합치기
    df_updated = pd.concat([df_old, df_new], ignore_index=True)

    # 중복 제거(더블 체크)
    df_updated = df_updated.drop_duplicates(subset=["ticker","link"], keep="first")

    # 저장
    df_updated.to_csv(csv_path, index=False, encoding="utf-8-sig")

    print(f"[{query}] 새로운 링크 {len(unique_links)}개 추가됨. 전체 {len(df_updated)}개 저장됨.")
    return df_updated

In [ ]:
# 3. 추출 대상에서 제외된 url을 csv로 저장해두는 함수
def save_filtered(query, link, filtered_path="yahoo_filtered.csv"):
    # 기존 파일 불러오기
    if os.path.exists(filtered_path):
        df_filtered = pd.read_csv(filtered_path)
    else:
        df_filtered = pd.DataFrame(columns=["ticker", "link"])

    # 새로운 행 추가
    new_row = pd.DataFrame([{"ticker": query, "link": link}])
    df_filtered = pd.concat([df_filtered, new_row], ignore_index=True)

    # 중복 제거
    df_filtered = df_filtered.drop_duplicates(subset=["ticker", "link"], keep="first")

    # 저장
    df_filtered.to_csv(filtered_path, index=False, encoding="utf-8-sig")
    print(f"추출 대상에서 제외된 url 저장: {link}")

In [ ]:
# 4. 기사 크롤링 함수
def scrape_articles(df, query, driver, csv_path="yahoo_news.csv"):

    # 재실행 시 index 불일치 해결을 위함
    i = 0
    while i < len(df):
        link = df.iloc[i]["link"]

        # 이미 headline 있으면 skip
        if "headline" in df.columns and pd.notnull(df.iloc[i]["headline"]):
            i += 1
            continue

        driver.get(link)
        time.sleep(2)

        try:
            # 필터링 조건1: PREMIUM News
            head_str = driver.find_element(By.XPATH, '//*[@id="main-content-wrapper"]')
            is_premium = head_str.text.split('\n', 1)[0].strip()
            if is_premium == "PREMIUM":
                print(f"Skip PREMIUM article: {link}")
                save_filtered(query, link)
                df = df.drop(df.index[i]).reset_index(drop=True)
                continue
            # 필터링 조건2: Yahoo Finance Video
            try:
                head_str_2 = driver.find_element(By.CLASS_NAME, 'byline-attr-author.yf-1k5w6kz')
                if 'Yahoo Finance Video' in head_str_2.text:
                    print(f"Skip Yahoo Finance VIDEO article: {link}")
                    save_filtered(query, link)
                    df = df.drop(df.index[i]).reset_index(drop=True)
                    continue
            except NoSuchElementException:
                pass

            # 필터링 조건3: 다양한 출처들(개인적인 견해를 포함한 글)
            try:
                element = driver.find_element(
                    By.CSS_SELECTOR,
                    'div.cover-wrap div.top-header a.subtle-link[aria-label]'
                )
                aria_label_value = element.get_attribute('aria-label')
                blocked_sources = [
                    'Motley Fool', 'Barchart', 'The Wall Street Journal',
                    'Zacks', 'Benzinga', 'Quartz'
                ]
                if any(src in aria_label_value for src in blocked_sources):
                    print(f"Skip SOURCE article ({aria_label_value}): {link}")
                    save_filtered(query, link)
                    df = df.drop(df.index[i]).reset_index(drop=True)
                    continue
            except NoSuchElementException:
                pass

            # 기사 제목
            headline = driver.find_element(By.CLASS_NAME, 'cover-headline.yf-1rjrr1').text
            # 발행일
            pubdate = driver.find_element(By.CLASS_NAME, 'byline-attr-meta-time').text
            # 관련 티커 (없는 경우 존재 -> None)
            try:
                tickers = driver.find_element(By.CLASS_NAME, 'carousel-top').text
            except NoSuchElementException:
                tickers = None
            # 기사 본문
            article = driver.find_element(By.CLASS_NAME, 'bodyItems-wrapper').text

            # 더보기 버튼 존재 시 클릭 -> 추가 본문
            try:
                continue_button = driver.find_element(By.CSS_SELECTOR, "button[aria-label='Story Continues']")
                continue_button.click()
                time.sleep(1)
                # 추가 본문
                add_article = driver.find_element(By.CLASS_NAME, 'read-more-wrapper').text
                # 본문 합치기
                article += "\n" + add_article
            except NoSuchElementException:
                pass

            # df 업데이트
            df.loc[i, "ticker"] = query
            df.loc[i, "headline"] = headline
            df.loc[i, "pubdate"] = pubdate
            df.loc[i, "related_tickers"] = tickers
            df.loc[i, "article"] = article

            # 정상 처리됐으면 i 증가
            i += 1

        except Exception as e:
            print(f"Error on {link}: {e}")
            # 에러 난 행도 삭제 (다시 시도하지 않음)
            df = df.drop(df.index[i]).reset_index(drop=True)
            # i 증가 안 함 → drop했으니 다음 행이 자동으로 i 위치에 옴

    # 저장 후 종료
    df.to_csv(csv_path, index=False, encoding="utf-8-sig")
    print(f"[{query}] 크롤링 완료 후 저장됨.")

In [ ]:
# 5. 결측치 제거 함수
def cleanup_csv(csv_path="yahoo_news.csv"):
    if os.path.exists(csv_path):
        df = pd.read_csv(csv_path)
        mask = df[["headline", "pubdate", "article"]].isnull().all(axis=1)
        df = df.drop(df.index[mask]).reset_index(drop=True)
        df.to_csv(csv_path, index=False, encoding="utf-8-sig")
        print(f"Null 행 {mask.sum()}개 삭제 완료")

In [ ]:
# 6. 모든 티커에 대해서 크롤링 실행
if __name__ == "__main__":

    # 실행 전 클린업 (링크만 있고 기사 데이터 없는 행 제거)
    csv_path = "yahoo_news.csv"
    cleanup_csv(csv_path)

    chrome_options = Options()
    chrome_options.add_argument('--headless')
    driver = webdriver.Chrome(options=chrome_options)

    nasdaq100 = get_nasdaq100_tickers(yahoo_style=True)

    try:
        for ticker in nasdaq100:
            df = update_csv(ticker, csv_path=csv_path, filtered_path="yahoo_filtered.csv")
            scrape_articles(df, ticker, driver, csv_path=csv_path)
            time.sleep(random.uniform(3, 6))  # 과부하 방지
    finally:
        driver.quit()
        print("모든 크롤링 완료 후 브라우저 종료")

In [ ]:
df.shape